In [ ]:
#!pip install flask

# não compartilhem a base de dados envolvidas nesse projeto

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from flask import Flask
from flask import request, jsonify
from flask import json, Response

import settings_flask

import pickle
import pandas as pd
import re
import spacy


In [ ]:
# CARREGA VARIÁVEIS DO PROJETO

# CRIA APP DA API
app = Flask(__name__)

# OBJETOS DOS MODELOS
nlp = spacy.load('pt_core_news_sm')
novo_vetor = pickle.load(open('modelo_verorizacao.pkl', 'rb'))
novo_modelo = pickle.load(open('modelo_classificacao.pkl', 'rb'))


In [ ]:
# função de lematização completa do documento
def fn_lematiza_texto(texto):
    sent = []
    doc = nlp(texto)
    for word in doc:
        sent.append(word.lemma_)
        
    return ' '.join(sent)

# função para limpar documento
def fn_limpa_texto(texto):
    texto = re.sub(r'@\w+','',texto)
    texto = re.sub(r'#','',texto)
    texto = re.sub(r'RT[\s]+','',texto)  
    #texto = re.sub(r'rt[\s]+','',texto)
    texto = re.sub(r'https:/\/\S+','',texto)
    texto = re.sub(r'[,.:;]','',texto)
    texto = re.sub(r'\n',' ',texto)
    texto = re.sub(u'[^a-zA-ZáéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ ]', '', texto)
    texto = texto.strip().lower()
    
    return texto

# função para fazer a predição do sentimento
def fn_sentimento(texto):
    texto = fn_limpa_texto(texto)
    texto = fn_lematiza_texto(texto)
    texto = [texto]
    text_vect = novo_vetor.transform(texto)
    texto_predito = novo_modelo.predict(text_vect)
    return texto_predito

In [ ]:
# INICIALIZA API
def initialize_app(flask_app):
    flask_app.config['SERVER_NAME'] = settings_flask.FLASK_SERVER_NAME
    flask_app.config['DEBUG'] = settings_flask.FLASK_DEBUG
    flask_app.config['JSON_AS_ASCII'] = settings_flask.FLASK_JSON_AS_ASCII


In [ ]:
# CRIA APP DA API
app = Flask(__name__)

# ROTA DA API - RECEBE ARG: TEXTO
@app.route('/api', methods=['GET'])
def fn_recebe_texto():    
    print('------------------------')
    print('requesição feita')
    print('----')    
    if 'texto' in request.args:        
        print('validou argumento')
        print('----')        
        texto = request.args['texto']        
        print('Texto requerido: ', texto)
        print('----')        
        if len(texto) > 0:            
            sentimento = fn_sentimento(texto)
            sentimento = sentimento[0]            
            print('sentimento predito: ', sentimento)
            print('----')            
            str_retorno = {'resultado':
                               {'texto': texto,
                                'sentimento': sentimento},
                            'status_geral':
                                {'status': 'Ok',
                                'mensagem': 'Sucesso'}}
        else:
            print('Erro: texto inválido')
            print('----')
            str_retorno = {'status_geral':
                        {'status': 'ERRO',
                        'mensagem': 'Requisição não contém um texto valido. Por favor, verifique sua requisição'}}
    else:
        print('Erro: requisição enválida')
        print('----')
        str_retorno = {'status_geral':
                        {'status': 'ERRO',
                        'mensagem': 'Requisição não contém o argumente de texto. Por favor, verifique sua requisição'}}
    
    print('fim da requisição e retorna resultado')
    print('------------------------')
    return jsonify(str_retorno)


In [ ]:
# COMPILA API
def main():
    initialize_app(app)
    app.run()

# EXECUTA API
if __name__ == "__main__":
    main()

**Teste da API**

In [ ]:
#http://localhost:8088/api?texto=que dia bonito!!!
#http://localhost:8088/api?texto=Filme horrível